_To use this notebook you must have a (free) FEO account, available [here](https://feo.transitionzero.org/). You may then install the feo-client library, if necessary, and then use the package to log in. You can also run notebook in a Colab notebook via the badge below._

<a target="_blank" href="https://colab.research.google.com/github/transition-zero/feo-client-examples/blob/main/feo-client-examples/5_land_cover.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip install feo-client

In [ ]:
from feo.client.auth import login

login()

# Land Cover Data
_Object-oriented interface for Land Cover Data via the Python client_

## Land Availability for Renewable Energy

The amount of land available to build renewables is a key constraint on the energy transition and hence a key input to energy system models. Understanding where and how much of a given technology can be built, constrains how renewable capacity can be deployed and has downstream effects on the cost and emissions of the energy system. If we want to understand the energy transition, getting land availability right matters!

To highlight this, let's do a back of the envelope calculation to see how land availability impacts how much solar PV we can be deployed in the Indonesian province Jawa Tengah. We'll consider two scenarios. In the first scenario, 1% of land in Jawa Tengah is available for solar PV. In the second, only 0.05% is available.

In [ ]:
# Assumptions
IDN_JT_land_area = 34290  # km2
scenario_1_percent = 0.01
scenario_2_percent = 0.007
solar_pv_density = 141  # MW/km2

# Compute IDN JT solar PV capacity in MW
idn_jt_solar_pv_capacity_scenario_1 = (
    IDN_JT_land_area * solar_pv_density * scenario_1_percent
)
idn_jt_solar_pv_capacity_scenario_2 = (
    IDN_JT_land_area * solar_pv_density * scenario_2_percent
)
capacity_difference = abs(
    idn_jt_solar_pv_capacity_scenario_1 - idn_jt_solar_pv_capacity_scenario_2
)

# Print results
print(
    f"Jawa Tengah solar PV capacity scenario 1: {idn_jt_solar_pv_capacity_scenario_1/ 1000:.2f} GW"
)
print(
    f"Jawa Tengah solar PV capacity scenario 2: {idn_jt_solar_pv_capacity_scenario_2/ 1000:.2f} GW"
)
print(f"Capacity difference: {capacity_difference/1000:.2f} GW")

We see a difference of 14.50 GW of solar PV capacity between the two scenarios! 

To put this in context, IRENA estimates that the UK currently has 14.4 GW of solar PV capacity. So, the difference between these two scenarios is roughly the current solar PV capacity of the UK. This is a huge difference and highlights the importance of getting land availability right.

## Land Cover Data in FEO

A key input to land availability calculations is land cover data. Land cover data tells us what type of land is present in a given location. 

In FEO, we have aggregated land cover data into $5$ land type categories:

- `urban`: Urban areas
- `cropland`: Agricultural land
- `forest`: Forested land
- `bare`: Bare land
- `shrub`: Shrub land + Herbaceous land

These land categories are derived from Coperniucs Global Land Cover classes

The area of each land type category can be accessed with the `RecordCollection` client at the node level.

In [ ]:
from feo.client import RecordCollection

forest_area = RecordCollection.search(node_id="IDN-JT", datum_detail="forest_landuse")
forest_area

The client returns a pandas dataframe and the area can be found in the `value` column. Looking at the `unit` column we see it is given in $km^2$.

Let's compare how much crop land there is in Jawa Tengah compared to forest..

In [ ]:
RecordCollection.search(node_id="IDN-JT", datum_detail="crop_landuse")

In [ ]:
IDN_JT_forest = forest_area["value"].item()
IDN_JT_crop = RecordCollection.search(node_id="IDN-JT", datum_detail="crop_landuse")[
    "value"
].item()

print(f"Jawa Tengah forest area: {IDN_JT_forest:.2f} km2")
print(f"Jawa Tengah cropland area: {IDN_JT_crop:.2f} km2")

.. and how much forest there is in Jawa Tengah, compared to a neighboring province, Jawa Barat.

In [ ]:
IDN_JB_forest = RecordCollection.search(
    node_id="IDN-JB", datum_detail="forest_landuse"
)["value"].item()

print(f"Jawa Tengah forest area: {IDN_JT_forest:.2f} km2")
print(f"Jawa Barat forest area: {IDN_JB_forest:.2f} km2")

## Renewable Potentials


This land cover data can be used for early-stage mapping of land availability for renewables and to gain a rough estimate of renewable energy potentials at a given node. A key assumption for this estimation is the percentage of a given land type that we want to make available for a given technology. This should be informed by the land use priorities of the region, renewable resource information (how windy/sunny is the region?) and the land use requirements of the technology.

For example, let's estimate how changing our forest land use policy impacts how much solar PV capacity could be deployed in currently forested areas of Jawa Tengah. 

In [ ]:
# Assumptions

solar_pv_density = 141  # MW/km2
less_deforestation_percentage = 0.01  # 1% of forest landuse is used for solar PV
more_deforestation_percentage = 0.05  # 5% of forest landuse is used for solar PV

IDN_JT_forest_pv_capacity_1 = (
    forest_area["value"].item() * solar_pv_density * less_deforestation_percentage
)
IDN_JT_forest_pv_capacity_2 = (
    forest_area["value"].item() * solar_pv_density * more_deforestation_percentage
)
print(
    f"If {less_deforestation_percentage*100}% of forest is used for solar PV in Jawa Tengah, then we can install {IDN_JT_forest_pv_capacity_1/1000:.2f} GW of solar PV in currently forested areas of Jawa Tengah."
)
print(
    f"If we use {more_deforestation_percentage*100}%, we can install {IDN_JT_forest_pv_capacity_2/1000:.2f} GW."
)

## Coming Soon ....

Currently FEO is in it's beta phase and we are working on adding new features. In the future we will add more detailed land use information, such as the land area available for a given technology. This will be based on desk research that informs "sensible defaults" for land use policy in different regions and also incorporates renewable resource data (e.g. how sunny/windy a given node is). Providing this information will allow users to quickly estimate the renewable potential of a given node, without having to do the calculations themselves.